## 1 模型预测控制（MPC）

### *参考资料
> - https://zhuanlan.zhihu.com/p/480031894
> - https://blog.csdn.net/u013468614/article/details/103519721
> - [Model predictive speed and steering control](https://atsushisakai.github.io/PythonRobotics/modules/path_tracking/model_predictive_speed_and_steering_control/model_predictive_speed_and_steering_control.html)

### <font color='red'>1.1 MPC实现步骤</font>

假设我们的模型一般格式为<font color='red'>(非线性dynamic可用Taylor展开并取线性项)
</font>

$$
X_{t+1} = A_t\cdot X_t + B_t\cdot U_t + C_t
$$

我们引入新的变量$\tilde{X}_t = [X_t, 1]^T$，则可以得到下面公式：

$$
\tilde{X}_{t+1} = \tilde{A}_t\cdot \tilde{X}_t +\tilde{B}_t\cdot U_t
$$
其中
$$
\tilde{A}_t = \left[
\begin{array}{cc}
A_t & C_t \\
0 & 1 \\
\end{array}
\right], 
\quad
\tilde{B}_t = 
\left[
\begin{array}{c}
B_t \\
0
\end{array}
\right]
$$

为了方便推导，我们依然将上诉公式记为

$$
X_{t+1} = A_t\cdot X_t +B_t\cdot U_t
$$

#### <font color='blue'>第一步：模型离散化与增量格式转换</font>
- 模型离散化：
假设离散时间步长为$dt$，则有
$$
\begin{align}
\ & X_{t+1} - X_t = dt \cdot (A_t\cdot X_t + B_tU_t) 
\\
\Longrightarrow\  & X_{t+1} = (I+ dt \cdot A_t) \cdot X_t + dt\cdot B_tU_t
\\
\Longrightarrow\  & \color{red}{X_{t+1} = \tilde{A}_t \cdot X_t + \tilde{B}_t\cdot U_t}
\end{align} 
$$
其中$\tilde{A}_t = I + dt\cdot A_t$，$\tilde{B}_t = dt\cdot B_t$。


- 增量格式转换：
利用增量控制取代当前的控制量，可以对控制量进行直接控制，也能防止计算过程中没有可行解的情况。
设新的状态变量为$\tilde{X}(k|t) = [X(k|t), U(k-1|t)]^T$，由于

$$
\begin{cases}
X(k+1) &= A(k)X(k) + B(k)U(k)  \\
       &= A(k)X(k) + B(k)\bigg(U(k-1)+\Delta U(k)\bigg) 
\\
U(k) & = U(k-1) + \Delta U(k)
\end{cases}
$$
我们可以得到
<font color='red'>
$$
\tilde{X}(k+1|t) = \tilde{A}(k|t)\cdot \tilde{X}(k|t) +\tilde{B}(k|t)\cdot \Delta U(k|t)
$$
</font>
其中
$$
\tilde{A}(k|t) = \left[
\begin{array}{cc}
A(k|t) & B(k|t) \\
0 & I \\
\end{array}
\right], 
\quad
\tilde{B}_t = 
\left[
\begin{array}{c}
B(k|t) \\
I
\end{array}
\right]
$$

#### <font color='blue'>第二步：方程推导</font>
假设预测区间为$\color{red}{P}$，控制区间为$\color{red}{N}$（$N\leq P$)，则我们对离散方程进行推导可得

$$
\begin{cases}
\tilde{X}(k+1|t) 
& = \tilde{A}_{k|t}\cdot \tilde{X}(k|t) + \tilde{B}_{k|t}\cdot\Delta U(k|t)
\\
\tilde{X}(k+2|t) 
& = \tilde{A}_{k+1|t}\cdot \tilde{X}(k+1|t) + \tilde{B}_{k+1|t}\cdot\Delta U(k+1|t)
\\
\vdots
\\
\tilde{X}(k+N|t) 
& = \tilde{A}_{k+N-1|t}\cdot \tilde{X}(k+N-1|t) + \tilde{B}_{k+N-1|t}\cdot\Delta U(k+N-1|t)
\\
\vdots \color{red}{(之后不再有新的\Delta U)}
\\
\tilde{X}(k+P|t) 
& = \tilde{A}_{k+P|t}\cdot \tilde{X}(k+P-1|t)
\end{cases}
$$

可以得到如下推导式：

$$
\begin{cases}
\tilde{X}(k+i) &= \tilde{AA}_{i}\cdot \tilde{X}(k) + \tilde{AB}^{(i)}_1\Delta U(k) + \cdots + \tilde{AB}^{(i)}_i\Delta U(k+i-1),\ \text{当} 1\leq i\leq N\\
\tilde{X}(k+i) &= \tilde{AA}^*_{i}\cdot \tilde{X}(k) + \tilde{AB}^{*(i)}_1\Delta U(k) + \cdots  + \tilde{AB}^{*(i)}_N\Delta U(k+N-1),\ \text{当} N<i\leq P
\end{cases}
$$

其中
$$
\text{当} (1\leq i\leq N):
\begin{cases}
\tilde{AA}_i &= \tilde{A}_{k+i-1}\tilde{A}_{k+i-2}\cdots\tilde{A}_{k} \\
\tilde{AB}^{(i)}_1 &= \tilde{A}_{k+i-1}\tilde{A}_{k+i-2}\cdots\tilde{A}_{k+1}\tilde{B}_{k}\\
\tilde{AB}^{(i)}_2 &= \tilde{A}_{k+i-1}\tilde{A}_{k+i-2}\cdots\tilde{A}_{k+2}\tilde{B}_{k+1} \\
\vdots \\
\tilde{AB}^{(i)}_{i-1} &= \tilde{A}_{k+i-1}\tilde{B}_{k+i-2} \\
\tilde{AB}^{(i)}_i &= \tilde{B}_{k+i-1} 
\end{cases}
$$

$$
\text{当} (N< i\leq P):
\begin{cases}
\tilde{AA}^*_i &= \tilde{A}_{k+i-1}\cdots\tilde{A}_{k+N}\tilde{A}_{k+N-1}\cdots\tilde{A}_{k} \\
\tilde{AB}^{*(i)}_1 &= \tilde{A}_{k+i-1}\cdots\tilde{A}_{k+N}\tilde{A}_{k+N-1}\cdots\tilde{A}_{k+1}\tilde{B}_{k}\\
\tilde{AB}^{*(i)}_2 &= \tilde{A}_{k+i-1}\cdots\tilde{A}_{k+N}\tilde{A}_{k+N-1}\cdots\tilde{A}_{k+2}\tilde{B}_{k+1} \\
\vdots \\
\tilde{AB}^{*(i)}_{N-1} &= \tilde{A}_{k+i-1}\cdots\tilde{A}_{k+N}\tilde{A}_{k+N-1}\tilde{B}_{k+N-2} \\
\tilde{AB}^{*(i)}_N &= \tilde{A}_{k+i-1}\cdots\tilde{A}_{k+N}\tilde{B}_{k+N-1} 
\end{cases}
$$

我们记
$$
Y(t) =
\left[
\begin{array}{c}
\tilde{X}(k+1|t)\\
\tilde{X}(k+1|t)\\
\vdots \\
\tilde{X}(k+P|t) 
\end{array}
\right]
,\quad 
\Delta \tilde{U}(t) =
\left[
\begin{array}{c}
\Delta U(k|t) \\
\Delta U(k+1|t) \\
\vdots \\
\Delta U(k+N-1|t)
\end{array}
\right]
$$

则有公式
<font color='red'>
$$
Y(t) = AA_t\cdot \tilde{X}(k|t) + BB_t\cdot \Delta\tilde{U}(t)
$$
</font>
其中
$$
AA_t = 
\left[
\begin{array}{c}
\tilde{AA}_1 \\
\vdots \\
\tilde{AA}_N \\
\tilde{AA}^*_{N+1} \\
\vdots   \\
\tilde{AA}^*_P 
\end{array}
\right]
,\quad
BB_t =
\left[
\begin{array}{cccc}
\tilde{AB}^{(1)}_1 &      0       & \cdots & 0 \\
\tilde{AB}^{(2)}_1 & \tilde{AB}^{(2)}_2 & \cdots & 0 \\
\vdots       &   \vdots     & \ddots & \vdots\\
\tilde{AB}^{(N)}_1 & \tilde{AB}^{(N)}_2 & \cdots & \tilde{AB}^{(N)}_N \\
\tilde{AB}^{*(N+1)}_1 &   \tilde{AB}^{*(N+1)}_2  & \cdots & \tilde{AB}^{*(N+1)}_N \\
\tilde{AB}^{*(N+2)}_1 &   \tilde{AB}^{*(N+2)}_2  & \cdots & \tilde{AB}^{*(N+2)}_N\\
\vdots       &   \vdots     & \ddots & \vdots\\
\tilde{AB}^{*(P)}_1 &   \tilde{AB}^{*(P)}_2  & \cdots & \tilde{AB}^{*(P)}_N\\
\end{array}
\right]
$$
> 注释：$BB_t$矩阵可以看成是MPC中当控制区间也为$P$时的$BB_t$矩阵的前$N$列构成的矩阵。

#### <font color='blue'>第三步：形成优化问题 </font>
假设目标函数为：
$$
J(\tilde{X}(t),\Delta U(t))=
\sum^{P}_{i}\|\tilde{X}(k+i|t) - \tilde{X}^d(k+i|t)\|^2_Q 
+ \sum^{N-1}_{i=0}\|\Delta U(k+i|t)\|^2_R
$$
> 注释：注意到$\tilde{X}(t)$中包含了$X(t)$和$U(t)$.

约束条件为：
$$
\begin{cases}
\color{red}{(1):}\ X_{min}(k+i|t)\leq X(k+i|t)\leq X_{max}(k+i|t),\quad i=1,\cdots,P 
\\
\\
\color{red}{(2):}\ U_{min}(k+i|t)\leq U(k+i|t)\leq U_{max}(k+i|t),\quad i=0,1,\cdots,N-1
\\
\\
\color{red}{(3):}\ \Delta U_{min}(k+i|t)\leq \Delta U(k+i|t)\leq \Delta U_{max}(k+i|t),\quad i=0,1,\cdots,N-1 
\end{cases}
$$

记$E_t=AA_t\tilde{X}_t - Y^d_t$，我们将其转化为二次规划问题

$$
\begin{align}
J & = (AA_t\tilde{X}_t+BB_t\Delta U_t - Y^d_t)^TQ(AA_t\tilde{X}_t+BB_t\Delta U_t - Y^d_t) \\
 &\quad + \Delta U_t^T R \Delta U_t
\\
\\
& = E_t^T Q E_t + 2E_t^T Q BB_t\Delta U_t +\Delta U_t^T (BB_t^TQ BB_t+R)\Delta U_t
\\
\\
& = \Delta U_t^T \color{red}{G_t}\Delta U_t + \color{red}{a_t}\Delta U_t + \color{red}{常数}
\end{align}
$$

其中
$$
\color{red}{G_t} = BB_t^TQ BB_t+R, \quad 
\color{red}{a_t} = 2E_t^T Q BB_t = 2(AA_t\tilde{X}_t - Y^d_t)^T Q BB_t
$$

#### <font color='blue'>第四步：处理约束条件</font>

-<font color='red'>处理约束条件(1)：</font> 
关于$Y_t$的约束可以变为

$$
Y_{min}\leq AA_t\tilde{X}(k|t) + BB_t \Delta U_t \leq Y_{max}
$$

因此可以转化为关于$\Delta U_t$的约束条件
$$
\begin{cases}
\color{red}{BB_t\Delta U_t \leq Y_{max} - AA_t\tilde{X}(k|t)} \\
\color{red}{-BB_t\Delta U_t \leq -Y_{min} + AA_t\tilde{X}(k|t)}
\end{cases}
$$


-<font color='red'>处理约束条件(2)：</font>
由于$U(k+i|t) = U(k+i-1|t) + \Delta U(k+i|t)$，因此有
$$
\left[
\begin{array}{c}
U_{k|t} \\
U_{k+1|t} \\
\vdots \\
U_{k+N-1|t} \\
\end{array}
\right] 
=
\left[
\begin{array}{c}
U_{t-1} \\
U_{t-1} \\
\vdots \\
U_{t-1} \\
\end{array}
\right]
+
\left[
\begin{array}{c}
\Delta U_{k|t} \\
\Delta U_{k|t} +\Delta U_{k+1|t} \\
\vdots \\
\sum^{N-1}_{i=0}\Delta U_{k+i|t} \\
\end{array}
\right]
$$

我们记
$$
C_u =
\left[
\begin{array}{cccc}
1 & 0 & \cdots & 0 \\
1 & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
1 & 1 & \cdots & 1
\end{array}
\right]
$$

则约束条件(1)可以转化为:
$$
\begin{cases}
\color{red}{C_u \Delta U_t \leq U_{max} - U_{t-1}} \\
\color{red}{- C_u \Delta U_t \leq - U_{min}+ U_{t-1} }
\end{cases}
$$


-<font color='red'>处理约束条件(3)：</font>
第三个约束条件无需做特别修改，即

$$
\begin{cases}
\color{red}{\Delta U_t \leq \Delta U_{max}} \\
\color{red}{-\Delta U_t \leq  -\Delta U_{min}}
\end{cases}
$$


综合之，可以得到以下条件：
<font color='red'>
$$
\left[
\begin{array}{c}
BB_t \\
-BB_t \\
C_u \\
- C_u  \\
I \\
-I 
\end{array}
\right]
\Delta U_t \leq
\left[
\begin{array}{c}
Y_{max} - AA_t\tilde{X}(k|t) \\
-Y_{min} + AA_t\tilde{X}(k|t) \\
U_{max} - U_{t-1} \\
- U_{min}+ U_{t-1} \\
\Delta U_{max} \\
-\Delta U_{min}
\end{array}
\right]
$$
</font>

#### <font color='blue'>第五步：求解优化问题</font>
综合第三步和第四步，我们可以得到如下的二次规划问题

$$
\begin{align}
J &=  \Delta U_t^T \color{red}{G_t}\Delta U_t + \color{red}{a_t}\Delta U_t + \color{red}{常数} \\
s.t. &\quad 
\left[
\begin{array}{c}
BB_t \\
-BB_t \\
C_u \\
- C_u  \\
I \\
-I 
\end{array}
\right]
\Delta U_t \leq
\left[
\begin{array}{c}
Y_{max} - AA_t\tilde{X}(k|t) \\
-Y_{min} + AA_t\tilde{X}(k|t) \\
U_{max} - U_{t-1} \\
- U_{min}+ U_{t-1} \\
\Delta U_{max} \\
-\Delta U_{min}
\end{array}
\right]
\end{align}
$$

求解该问题得到最优解$\Delta U^{opt}_t$，取第一项$\color{red}{\Delta U^{opt}(k|t)}$作为实际控制增量，并得到下一步的控制

$$
\color{red}{U(t) = U(t-1) + \Delta U^{opt}(k|t)}
$$

## 4. 数值算例：自动驾驶小车 
考虑简化车辆运动模型如下（不考虑控制加速度a）：
$$
\begin{cases}
\dot{x} &=  v\cos(\psi) \\
\dot{y} &= v\sin(\psi) \\
\dot{\psi} &= \frac{v}{L}\tan(\delta) 
\end{cases}
$$

#### 模型线性化：
$$
\begin{equation}
\color{blue}{(*): }
\dot{X} =
\left[ \begin{array}{cc}
        \dot{x} \\
        \dot{y} \\
        \dot{\psi} 
        \end{array}
\right] = 
\left[ \begin{array}{cc}
        v\cos(\psi) \\
        v\sin(\psi) \\
        \frac{v\tan{\delta}}{L} 
        \end{array}
\right] 
= f(X,u)
\end{equation}
$$
其中$X=[x,y,\psi]^T,\ u=[v, \delta]^T$.

将$(*)$在$X_r=[x_r,y_r,\psi_r]^T, u_r=[v_r, \delta_r]^T$处展开并忽略二阶和高阶项，可得：
$$
\begin{align}
\dot{X} = f(X,u) &\approx f(X_r, u_r) + \frac{\partial f}{\partial X_r}(X-X_r) 
+ \frac{\partial f}{\partial u_r}(u-u_r) \\
& = \dot{X}_r + A_t(X-X_r) + B_t(u-u_r)
\end{align}
$$
其中
$$
A_t =
\left[
\begin{array}{cccc}
0 & 0 & -v_r\sin{\psi_r}\\
0 & 0 & v_r\cos{\psi_r}\\
0 & 0 & 0 
\end{array}
\right],
\quad
B_t =
\left[
\begin{array}{cc}
\cos(\psi_r) & 0  \\
\sin(\psi_r) & 0  \\
\frac{\tan(\delta_r)}{L} & \frac{v_r}{L\cos^2{\delta_r}} \\
\end{array}
\right]
$$

记<font color='red'>$\Delta X=X-X_r,\ \Delta u =u-u_r$</font>, 则有
$$
\color{red}{(*):\quad \Delta \dot{X} = A_t\Delta X + B_t \Delta u}
$$

#### 线性模型离散化：
我们对$(*)$进行离散化，假设时间步长为$dt$，则有
$$
\begin{align}
\Delta X_{t+1} &= (I+dtA_t)\Delta X_t + dtB_t\Delta u_t \\
 &= \tilde{A}_t \Delta X_t + \tilde{B}_t \Delta u_t.
\end{align}
$$
其中
$$
\tilde{A}_t = I+dtA_t 
=\left[
\begin{array}{cccc}
1 & 0 & -v_r\sin{\psi_r}dt \\
0 & 1 & v_r\cos{\psi_r}dt \\
0 & 0 & 1 & \\
\end{array}
\right]
,
\quad \tilde{B}_t = dtB_t 
=\left[
\begin{array}{cc}
\cos{\psi_r}dt & 0  \\
\sin{\psi_r}dt& 0  \\
\frac{\tan{\delta_r}}{L}dt & \frac{v_r}{L\cos^2{\delta_r}}dt 
\end{array}
\right]
.
$$

#### 模型展开：
$$
\begin{align}
X_{t+1} 
&= X^r_{t+1}+ \tilde{A}_t (X_t - X^r_t)+ \tilde{B}_t (u_t - u^r_t) \\
& = \tilde{A}_t X_t + \tilde{B}_t u_t + (X^r_{t+1} - \tilde{A}_t X^r_t - \tilde{B}_t u^r_t) \\
& = \tilde{A}_t X_t + \tilde{B}_t u_t + (X^r_{t} + f(x_r,u_r)dt - X^r_t -A_tdt X^r_t - B_td_t u^r_t) \\
& = \tilde{A}_t X_t + \tilde{B}_t u_t + (f(x_r,u_r)  -A_t X^r_t - B_t u^r_t)dt
\end{align} 
$$

即，得到
$$
X_{t+1} =
\tilde{A}_t X_t + \tilde{B}_t u_t + C
$$
其中
$$
C =
\bigg(
\left[ \begin{array}{c}
        v^r\cos(\psi^r) \\
        v^r\sin(\psi^r) \\
        \frac{v^r\tan{\delta^r}}{L} 
        \end{array}
\right] 
- 
\left[
\begin{array}{c}
 -v_r\psi_r\sin{\psi_r}\\
v_r\psi_r\cos{\psi_r}\\
0 
\end{array}
\right]
-
\left[
\begin{array}{c}
v_r\cos(\psi_r)  \\
v_r\sin(\psi_r) \\
\frac{v_r\tan(\delta_r)}{L} + \frac{v_r\delta_r}{L\cos^2{\delta_r}}\\
\end{array}
\right]
\bigg)dt
=
\left[
\begin{array}{c}
 v_r\psi_r\sin{\psi_r}dt\\
-v_r\psi_r\cos{\psi_r}dt\\
-\frac{v_r\delta_r}{L\cos^2{\delta_r}}dt
\end{array}
\right]
$$

### 4.1: 路径追踪问题

模型方程为：
$$
X_{t+1} =
\tilde{A}_t X_t + \tilde{B}_t u_t + C
$$
其中
$$
\tilde{A}_t
=\left[
\begin{array}{cccc}
1 & 0 & -v_r\sin{\psi_r}dt \\
0 & 1 & v_r\cos{\psi_r}dt \\
0 & 0 & 1 & \\
\end{array}
\right]
,
\  \tilde{B}_t
=\left[
\begin{array}{cc}
\cos{\psi_r}dt & 0  \\
\sin{\psi_r}dt& 0  \\
\frac{\tan{\delta_r}}{L}dt & \frac{v_r}{L\cos^2{\delta_r}}dt 
\end{array}
\right]
,
\  C
=
\left[
\begin{array}{c}
 v_r\psi_r\sin{\psi_r}dt\\
-v_r\psi_r\cos{\psi_r}dt\\
-\frac{v_r\delta_r}{L\cos^2{\delta_r}}dt
\end{array}
\right].
$$

我们设置$u_r = [v_r, \delta_r] = [2., 0.]$，但其中的<font color='red'>$\tilde{A}_t, \tilde{B}_t, \tilde{C}_t$依赖于$X_r$中的$\psi_r$</font>.

### 4.2: 代码实现
> 代码实现参见 uav_car_simple_mpc.py